In [1]:
import numpy as np
import pandas as pd

import importlib
import TechnicalAnalysisFeatures
importlib.reload(TechnicalAnalysisFeatures)
from TechnicalAnalysisFeatures import *

In [42]:
data_path = './data/'
train = pd.read_csv(data_path + 'train.csv', nrows = 10000) 

In [43]:
train['date'] = pd.to_datetime(train['timestamp'], unit = 's')
train = train.sort_values('date')
train.set_index(train['timestamp'], inplace=True)
train.drop(['timestamp'], axis = 1,inplace=True)

In [5]:
#Whether to load strict data or not
LOAD_STRICT = 0
if LOAD_STRICT: 
    train = train.loc[train['date'] < "2021-06-13 00:00:00"]    

In [46]:
#Select which asset
train = train[train["Asset_ID"] == 1]

In [49]:
#fill missing timestamp
train = train.reindex(
    range(train.index[0],train.index[-1]+60,60), method='pad'
)

In [72]:
Ns = [3, 5, 10, 30]

#parameters for KST
r1, r2, r3, r4 = 10, 15, 20, 30
n1, n2, n3, n4 = 10, 10, 10, 15

#parameter for MACD 
n_fasts = [3, 5]
n_slows = [10, 15]

#ADX
n_ADX = 10

for n in Ns:
    train = MA(train, n)
    train = EMA(train, n)
    train = MOM(train, n)
    train = ATR(train, n)
    train = BBANDS(train, n)
    train = TRIX(train, n)
    train = ADX(train, n, n_ADX)
    train = Vortex(train, n)
    train = RSI(train, n)
    train = ACCDIST(train, n)
    train = MFI(train, n)
    train = OBV(train, n)
    train = FORCE(train, n)
    train = EOM(train, n)
    train = CCI(train, n)
    train = COPP(train, n)
    train = KELCH(train, n)
    train = DONCH(train, n)
    train = STDDEV(train, n)

for n_fast in n_fasts:
    for n_slow in n_slows:
        train = MACD(train, n_fast, n_slow)

train = PPSR(train)
train = STOK(train)
train = MassI(train)
train = Chaikin(train)
train = ULTOSC(train)

train = KST(train, r1, r2, r3, r4, n1, n2, n3, n4)

# Feature selections

In [73]:
from sklearn.linear_model import LassoCV

In [74]:
SEED = 1126
drop_cols = ["Target", "Asset_ID", "date"]
keep_cols = list(
    set(train.columns) - set(drop_cols)
)
y_train = train["Target"]
X_train = train.loc[:, keep_cols]

In [75]:
#Lasso can't handle NaN
X_train_ = X_train.fillna(0)

In [76]:
clf = LassoCV(
    cv=5,random_state=SEED, max_iter = 50000
    ).fit(X_train_, y_train)

C:\Users\martin\.conda\envs\test\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.625332143806446e-05, tolerance: 4.432083740153707e-06
  tol, rng, random, positive)
C:\Users\martin\.conda\envs\test\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.621444004059087e-05, tolerance: 4.432083740153707e-06
  tol, rng, random, positive)
C:\Users\martin\.conda\envs\test\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.8259891566427553e-05, tolerance: 4.432083740153707e-06
  tol, rng, random, positive)
C:\Users\martin\.conda\envs\test\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472:

In [77]:
acc = clf.score(X_train_, y_train)
print(f"Training acc of best model = {round(acc * 100, 4)}%")
print("Cross Validation result in each parameter sets")
pd.DataFrame({"Alpha":clf.alphas_, "Cross valid MSE": clf.mse_path_.mean(axis = 1)})

Training acc of best model = 7.5851%
Cross Validation result in each parameter sets


,Alpha,Cross valid MSE
0,1.835950,0.000044
1,1.712213,0.000044
2,1.596816,0.000044
3,1.489196,0.000044
4,1.388829,0.000044
...,...,...
95,0.002427,0.001270
96,0.002263,0.001163
97,0.002111,0.001002
98,0.001969,0.000872


In [78]:
final_features_l1Reg = X_train_.columns[(clf.coef_ > 0).flatten()]
print(f"There are {len(final_features_l1Reg)} features selected by L1 regression")
print(final_features_l1Reg.tolist())

There are 4 features selected by L1 regression
['KelChD_10', 'Force_10', 'Force_5', 'Force_3']


## By correlation

In [19]:
#fill nan with mean to avoid bias
X_train_ = X_train.fillna(X_train.mean())
corr_value = X_train_.corrwith(y_train)
corr_abs = abs(corr_value).sort_values(ascending=False)

In [20]:
THRESHOLD = 0.05
final_features_corr = corr_abs[corr_abs > THRESHOLD].index
print(f"There are {len(final_features_corr)} features selected by correlation coefficient")
print(final_features_corr.tolist())

There are 10 features selected by correlation coefficient
['Open', 'VWAP', 'High', 'Close', 'EMA_3', 'MA_3', 'EMA_5', 'MA_5', 'Low', 'Volume']
